In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import datetime

# from bokeh.io import output_file, show
# from bokeh.models import ColumnDataSource, GMapOptions
# from bokeh.plotting import gmap

from PIL import Image, ImageDraw
import os
import shutil

from mpl_toolkits.basemap import Basemap

import FCM

%matplotlib inline

#Turn off interactive plotting for pyplot
plt.ioff()

In [ ]:
#Read in the data from the lightning file and the storm centered
df = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_Reduced_Trackfile.txt',header=None,names=["Year","Month","Day","Hour","Lat","Long","Min_Pressure","Max_Winds","Unused"],low_memory=False,sep='\t')
df = df.drop("Unused",axis=1)
ln = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_WWLLN_Locations.txt',header=None,names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"],low_memory=False,sep=' ')
ln['Distance'] = np.sqrt(ln['Dist_East_West'] ** 2 + ln['Dist_North_South'] ** 2)

# Map boundaries
minLong_, minLat_, maxLong_, maxLat_ = ln['Long'].min(), ln['Lat'].min(), ln['Long'].max(), ln['Lat'].max()
buffer = 10

In [ ]:
# Create interpolated data
dfc = df.copy()
for i in range(60):
    s = dfc.iloc[i]
    s2 = dfc.iloc[i+1]
    newLat = np.linspace(s.Lat,s2.Lat,num=6)
    newLong = np.linspace(s.Long,s2.Long,num=6)
    newMin = np.linspace(s.Min_Pressure,s2.Min_Pressure,num=6)
    newWinds = np.linspace(s.Max_Winds, s2.Max_Winds, num=6)
    newHours = np.arange(s.Hour,s.Hour + 6)
    newYears = [s.Year for x in range(6)]
    newMonths = [s.Month for x in range(6)]
    newDays = [s.Day for x in range(6)]
    newDf = pd.DataFrame([newYears, newMonths, newDays, newHours, newLat, newLong, newMin, newWinds]).T
    newDf.columns=['Year','Month','Day','Hour','Lat','Long','Min_Pressure','Max_Winds']
    dfc = dfc.append(newDf,ignore_index=True)

df, dfc = dfc, df # Swap the two dataframes

# The below cell graphs everything but without clustering

In [ ]:
# mi = 30 #Minute Intervals - How many minutes between frames

# #Graph the center of the storm and lightning
# for x in range(len(df['Month'].unique())):
#     month = df['Month'].unique()[x]
#     for y in range(len(df[df['Month'] == df['Month'].unique()[x]]['Day'].unique())):
#         day = df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]
#         for z in range(len(df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique())):
#             hour = df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique()[z]
#             for w in range(0,60,mi):
#                 minute = w
#                 #The above gets the month, day, hour, minute (before 30 minutes or after 30 minutes)
#                 #Make big map
#                 fig = plt.figure(figsize=(30,15))
#                 m = Basemap(llcrnrlon=minLong_-buffer, llcrnrlat=minLat_-buffer,urcrnrlon=maxLong_+buffer,urcrnrlat=maxLat_+buffer,lon_0=0,lat_0=0)
#                 m.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
#                 m.fillcontinents(color='grey', alpha=0.7, lake_color='grey')
#                 m.drawcoastlines(linewidth=0.1, color="white")
#                 #Plot ticks for lat/long
#                 plt.xticks(np.arange(minLong_-buffer,maxLong_+buffer,step=5))
#                 plt.yticks(np.arange(minLat_-buffer,maxLat_+buffer,step=5))
#                 #Title the map 2017:month:day::hour:minute
#                 plt.title("2017:" + '{:2d}'.format(int(month)) + ":" + '{:2d}'.format(int(day)) + "::" + '{:2d}'.format(int(hour)) + ":" + '{:2d}'.format(int(minute)))
#                 #Plot the center of Irma
#                 m.plot(df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour)]['Long'], df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour)]['Lat'], linestyle='none', marker="o", markersize=30, alpha=1, c="red", markeredgecolor="black", markeredgewidth=1)
#                 #Plot all of the lightning that appears on that month/day/hour/minute section
#                 m.plot(ln[(ln['Month'] == month) & (ln['Day'] == day) & (ln['Hour'] == hour) & (ln['Min'] >= minute) & (ln['Min'] <= minute + mi)]['Long'],ln[(ln['Month'] == month) & (ln['Day'] == day) & (ln['Hour'] == hour) & (ln['Min'] >= minute) & (ln['Min'] <= minute + mi)]['Lat'], linestyle='none', marker="X", markersize=8, alpha=.7, c="yellow", markeredgecolor="black", markeredgewidth=1)
#                 #Save and close the figure
#                 plt.savefig("./data/Irma/" + "2017_" + str(int(month)) + "_" + str(int(day)) + "_" + str(int(hour)) + "_" + str(int(minute)),bbox_inches='tight')
#                 plt.close(fig)

# The cell below graphs everything with clustering

In [ ]:
mi = 30 #Minute Intervals - How many minutes between frames

for x in range(len(df['Month'].unique())):
    month = df['Month'].unique()[x]
    for y in range(len(df[df['Month'] == df['Month'].unique()[x]]['Day'].unique())):
        day = df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]
        for z in range(len(df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique())):
            hour = df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique()[z]
            for w in range(0,60,mi):
                minute = w
                #The above gets the month, day, hour, minute (before 30 minutes or after 30 minutes)
                #Make big map
                fig = plt.figure(figsize=(30,15))
                m = Basemap(llcrnrlon=minLong_-buffer, llcrnrlat=minLat_-buffer,urcrnrlon=maxLong_+buffer,urcrnrlat=maxLat_+buffer,lon_0=0,lat_0=0)
                m.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
                m.fillcontinents(color='grey', alpha=0.7, lake_color='grey')
                m.drawcoastlines(linewidth=0.1, color="white")
                
                #Plot ticks for lat/long
                plt.xticks(np.arange(minLong_-buffer,maxLong_+buffer,step=5))
                plt.yticks(np.arange(minLat_-buffer,maxLat_+buffer,step=5))
                #Title the map 2017:month:day::hour:minute
                plt.title("2017:" + '{:2d}'.format(int(month)) + ":" + '{:2d}'.format(int(day)) + "::" + '{:2d}'.format(int(hour)) + ":" + '{:2d}'.format(int(minute)))
                
                # Make copies of all the dataframes to add the centroids onto
                ln_inner = ln[(ln['Month'] == month) & (ln['Day'] == day) & (ln['Hour'] == hour) & (ln['Min'] >= minute) & (ln['Min'] <= minute + mi) & (ln['Distance'] < 100)].copy()
                ln_middle = ln[(ln['Month'] == month) & (ln['Day'] == day) & (ln['Hour'] == hour) & (ln['Min'] >= minute) & (ln['Min'] <= minute + mi) & (ln['Distance'] >= 100) & (ln['Distance'] < 200)].copy()
                ln_rain = ln[(ln['Month'] == month) & (ln['Day'] == day) & (ln['Hour'] == hour) & (ln['Min'] >= minute) & (ln['Min'] <= minute + mi) & (ln['Distance'] >= 200) & (ln['Distance'] < 400)].copy()
                ln_outer = ln[(ln['Month'] == month) & (ln['Day'] == day) & (ln['Hour'] == hour) & (ln['Min'] >= minute) & (ln['Min'] <= minute + mi) & (ln['Distance'] >= 400)].copy()
                
                # Setup FCM to run on each dataframe
                num_cents = 5
                fcm_inner = FCM.FCM(ln_inner[['Long','Lat']],num_cents,2,maxiter=5,genCentroids=True)
                fcm_middle = FCM.FCM(ln_middle[['Long','Lat']],num_cents,2,maxiter=5,genCentroids=True)
                fcm_rain = FCM.FCM(ln_rain[['Long','Lat']],num_cents,2,maxiter=5,genCentroids=True)
                fcm_outer = FCM.FCM(ln_outer[['Long','Lat']],num_cents,2,maxiter=5,genCentroids=True)
                
                # Fit the data and generate the centroids
                cents_inner = fcm_inner.fit()
                cents_middle = fcm_inner.fit()
                cents_rain = fcm_inner.fit()
                cents_outer = fcm_inner.fit()
                
                # Classify each point with simple classification
                ln_inner['C'] = fcm_inner.classify()
                ln_middle['C'] = fcm_middle.classify()
                ln_rain['C'] = fcm_rain.classify()
                ln_outer['C'] = fcm_outer.classify()
                
                # Plot the center of Irma
                m.plot(df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour)]['Long'], df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour)]['Lat'], linestyle='none', marker="o", markersize=20, alpha=1, c="yellow", markeredgecolor="black", markeredgewidth=1,zorder=2)                
                
                # Plot all of the centroids               
                m.scatter(cents_inner['Long'],cents_inner['Lat'],marker='*',c=cents_inner.index/num_cents,s=90,cmap='spring',edgecolors='black',linewidths=1,zorder=4)
                m.scatter(cents_middle['Long'],cents_middle['Lat'],marker='*',c=cents_middle.index/num_cents,s=90,cmap='summer',edgecolors='black',linewidths=1,zorder=4)
                m.scatter(cents_rain['Long'],cents_rain['Lat'],marker='*',c=cents_rain.index/num_cents,s=90,cmap='autumn',edgecolors='black',linewidths=1,zorder=4)
                m.scatter(cents_outer['Long'],cents_outer['Lat'],marker='*',c=cents_outer.index/num_cents,s=90,cmap='winter',edgecolors='black',linewidths=1,zorder=4)
                
                # Plot all of the points
                m.scatter(ln_inner['Long'],ln_inner['Lat'],c=ln_inner['C']/num_cents,cmap='spring',s=45,edgecolors='black',linewidths=1,zorder=3)
                m.scatter(ln_middle['Long'],ln_middle['Lat'],c=ln_middle['C']/num_cents,cmap='summer',s=45,edgecolors='black',linewidths=1,zorder=3)
                m.scatter(ln_rain['Long'],ln_rain['Lat'],c=ln_rain['C']/num_cents,cmap='autumn',s=45,edgecolors='black',linewidths=1,zorder=3)
                m.scatter(ln_outer['Long'],ln_outer['Lat'],c=ln_outer['C']/num_cents,cmap='winter',s=45,edgecolors='black',linewidths=1,zorder=3)
                
                # Delete all of the dataframes (spare some sweet sweet ram)
                ln_inner = pd.DataFrame(None)
                ln_middle = pd.DataFrame(None)
                ln_rain = pd.DataFrame(None)
                ln_outer = pd.DataFrame(None)
                
                #Save and close the figure
                plt.savefig("./dump/" + "2017_" + str(int(month)) + "_" + str(int(day)) + "_" + str(int(hour)) + "_" + str(int(minute)),bbox_inches='tight')
                plt.close(fig)